# SUB-BENCHMARK3: Comparing jDR methods on single-cell datasets

The performances of the 9 jDR methods are here compared based on their ability to cluster cells based on their cancer cell line of origine. The clustering is performed jointly considering scRNA-seq and scATAC-seq data. 

## Data preprocessing
First the data are read in their original format and adapted to be read as input of our run_factorization function.

In [2]:
# Load data and processing

# Load RNA-seq data
exp <- readRDS("../data/single-cell/CellLines_RNAseqCounts.RDS", refhook = NULL) #ENS for genes and counts
# Apply log2 on RNA-seq data
exp <- log2(exp+1)
# Load ATAC-seq data
atac_counts<-readRDS("../data/single-cell/CellLines_ATACseqCounts.RDS", refhook = NULL) # peaks counts
# Load metadata
metadata<-readRDS("../data/single-cell/CellLines_metadata.RDS", refhook = NULL)
# Rename columns from metadata
colnames(atac_counts) <- metadata[,1]

# Export RNA-seq data as tab-separated table
write.table(exp, "../data/single-cell/CellLines_RNAseqCounts.txt", 
            sep="\t", col.names=TRUE, row.names=TRUE)
# Add a name ("probe") to the first column
system("sed -i '1s/^/probe\t/' ../data/single-cell/CellLines_RNAseqCounts.txt")
# Export ATAC-seq data as tab-separated table
write.table(atac_counts, "../data/single-cell/CellLines_ATACseqCounts.txt", 
            sep="\t", col.names=TRUE, row.names=TRUE)
# Add a name ("probe") to the first column
system("sed -i '1s/^/probe\t/' ../data/single-cell/CellLines_ATACseqCounts.txt")

In [5]:
exp

,HCT_CL100049490_L01_10,HCT_CL100049490_L01_14,HCT_CL100049490_L01_15,HCT_CL100049490_L01_16,HCT_CL100049490_L01_19,HCT_CL100049490_L01_2,HCT_CL100049490_L01_21,HCT_CL100049490_L01_22,HCT_CL100049490_L01_24,HCT_CL100049490_L01_27,⋯,K562_CL100050926_L02_79,K562_CL100050926_L02_81,K562_CL100050926_L02_83,K562_CL100050926_L02_84,K562_CL100050926_L02_85,K562_CL100050926_L02_86,K562_CL100050926_L02_87,K562_CL100050926_L02_88,K562_CL100050926_L02_92,K562_CL100050926_L02_93
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000003,4.754888,6.918863,5.807355,6.906891,6.599913,7.266787,6.857981,7.087463,7.179909,6.629357,⋯,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000000005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,⋯,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000000419,6.129283,6.409391,6.339850,6.409391,6.426265,6.169925,6.108524,6.614710,6.357552,6.189825,⋯,4.754888,4.807355,6.285402,5.044394,5.643856,5.857981,5.906891,6.129283,5.954196,6.409391
ENSG00000000457,6.584963,5.954196,7.209453,1.000000,5.209453,5.700440,7.209453,7.055282,2.321928,4.000000,⋯,6.539159,5.087463,0.000000,5.906891,3.906891,0.000000,6.000000,6.741467,3.906891,1.000000
ENSG00000000460,5.807355,6.087463,7.679480,1.000000,6.988685,7.294621,7.276124,6.988685,1.000000,6.339850,⋯,5.700440,3.169925,0.000000,4.000000,6.599913,1.000000,4.392317,6.942515,5.459432,4.247928
ENSG00000000938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,⋯,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000000971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,⋯,6.781360,6.285402,8.243174,8.134426,7.857981,7.507795,8.169925,7.794416,8.189825,8.169925
ENSG00000001036,7.044394,7.159871,7.129283,6.954196,6.988685,7.442943,7.011227,7.199672,6.954196,6.658211,⋯,6.392317,6.426265,6.942515,5.357552,6.781360,6.426265,7.044394,7.055282,7.276124,7.139551
ENSG00000001084,0.000000,3.584963,7.629357,1.000000,0.000000,6.741467,3.459432,6.129283,7.312883,0.000000,⋯,0.000000,0.000000,0.000000,0.000000,0.000000,3.584963,0.000000,0.000000,4.906891,0.000000


In [6]:
atac_counts

,HCT_CL100049490_L01_10,HCT_CL100049490_L01_14,HCT_CL100049490_L01_15,HCT_CL100049490_L01_16,HCT_CL100049490_L01_19,HCT_CL100049490_L01_2,HCT_CL100049490_L01_21,HCT_CL100049490_L01_22,HCT_CL100049490_L01_24,HCT_CL100049490_L01_27,⋯,K562_CL100050926_L02_79,K562_CL100050926_L02_81,K562_CL100050926_L02_83,K562_CL100050926_L02_84,K562_CL100050926_L02_85,K562_CL100050926_L02_86,K562_CL100050926_L02_87,K562_CL100050926_L02_88,K562_CL100050926_L02_92,K562_CL100050926_L02_93
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
HCT116_peak_1,0,0,1,0,0,1,0,0,0,0,⋯,3,1,1,2,1,0,1,2,0,0
Hela_peak_2,1,3,2,2,2,8,4,1,1,0,⋯,0,4,2,1,0,2,1,0,0,0
K562_peak_2,1,0,0,0,0,0,1,1,0,0,⋯,0,1,0,0,0,0,0,0,0,0
HCT116_peak_3,0,5,0,0,4,2,1,2,0,0,⋯,0,1,0,4,0,1,2,2,2,1
K562_peak_4,0,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,1,0,0,0,0
HCT116_peak_4,3,2,0,1,3,3,1,0,3,2,⋯,0,1,3,1,2,1,0,1,1,2
Hela_peak_5,0,0,0,0,0,2,0,0,0,0,⋯,0,0,0,0,0,0,2,1,0,0
Hela_peak_6,0,0,0,0,2,2,1,2,1,1,⋯,2,1,1,0,1,0,2,0,0,1
Hela_peak_7,0,0,0,0,0,0,0,0,0,0,⋯,1,0,0,0,0,0,0,0,0,0


In [7]:
metadata

,sample.rna,sample.atac,Total.Read.Pairs,Mapped.read.pairs,chrM.read.pairs,Unique.Flagments,Usable.Flagments,Flagments.in.peaks,Rate.of.Flagments.in.peaks...,lane.atac,⋯,unique.mapped.gene.,Multiple.mapped.gene.,Gene.mapped.rate...,Gene.number.FPKM..1.,ERCC.number,lane.rna.y,batch.rna.y,celltype,quality.pass,Type
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<chr>,⋯,<int>,<int>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<chr>
1,HCT_CL100049490_L01_10,HCT_CL100050215_L01_10,5652832,2142410,308401,288574,287830,43823,15.22531,CL100050215_L01,⋯,2428210,6764665,79.13,12728,57,CL100049490_L01,scCAT_20171127_HCT_R1_1.Summary.xls,HCT,1,HCT
2,HCT_CL100049490_L01_14,HCT_CL100050215_L01_14,7927709,2420888,227655,168073,167591,47288,28.21631,CL100050215_L01,⋯,2454685,6494370,74.84,12813,55,CL100049490_L01,scCAT_20171127_HCT_R1_1.Summary.xls,HCT,1,HCT
3,HCT_CL100049490_L01_15,HCT_CL100050215_L01_15,8299477,3220612,261155,295228,294344,60415,20.52530,CL100050215_L01,⋯,3394474,9403818,77.33,13510,55,CL100049490_L01,scCAT_20171127_HCT_R1_1.Summary.xls,HCT,1,HCT
4,HCT_CL100049490_L01_16,HCT_CL100050215_L01_16,7949453,2976918,181194,203974,203278,46737,22.99167,CL100050215_L01,⋯,3689905,8437816,76.62,11754,55,CL100049490_L01,scCAT_20171127_HCT_R1_1.Summary.xls,HCT,1,HCT
5,HCT_CL100049490_L01_19,HCT_CL100050215_L01_19,11619678,4256602,461415,440250,439264,92790,21.12397,CL100050215_L01,⋯,2606176,6962609,79.86,13875,55,CL100049490_L01,scCAT_20171127_HCT_R1_1.Summary.xls,HCT,1,HCT
6,HCT_CL100049490_L01_2,HCT_CL100050215_L01_2,15639248,6190197,491529,672421,670961,116239,17.32426,CL100050215_L01,⋯,4587203,11397784,78.55,13526,59,CL100049490_L01,scCAT_20171127_HCT_R1_1.Summary.xls,HCT,1,HCT
7,HCT_CL100049490_L01_21,HCT_CL100050215_L01_21,12207598,4472701,316767,310128,309360,55739,18.01752,CL100050215_L01,⋯,3625588,10465020,81.48,12897,54,CL100049490_L01,scCAT_20171127_HCT_R1_1.Summary.xls,HCT,1,HCT
8,HCT_CL100049490_L01_22,HCT_CL100050215_L01_22,13049794,5233571,695232,587671,585947,107482,18.34330,CL100050215_L01,⋯,3754738,9237028,76.17,14662,60,CL100049490_L01,scCAT_20171127_HCT_R1_1.Summary.xls,HCT,1,HCT
9,HCT_CL100049490_L01_24,HCT_CL100050215_L01_24,19661805,6938338,648426,234046,233199,52978,22.71794,CL100050215_L01,⋯,4246809,10543787,78.98,12993,57,CL100049490_L01,scCAT_20171127_HCT_R1_1.Summary.xls,HCT,1,HCT


## Running comparison
Two factor are then detected for each jDR method and the distribution of the cells with respect of Factor1 and Factor2 is plotted as a scatter plot. The obtained plots are available in the Results folder. The capability of the different jDR methods to cluster the cells accoridng to their cell line of origin is finally evaluated through the C-index, whose value is reported in the Results folder.

In [2]:
library("ggplot2")
library("clusterCrit")
source("runfactorization.R")

# Parameters for the plots
dot_size <- 1.5
dot_alpha <- 1.0
xlabel <- "Factor 1"
ylabel <- "Factor 2"

# Load annotations from the metadata
sample_annot <- metadata[, c("sample.rna", "celltype")]

# Folder for results
results_folder <- "../results_single_cell/"
# Create output folder
dir.create(results_folder, showWarnings = FALSE)

# Run factorization methods
out <- runfactorization("../data/single-cell/",
                        c("CellLines_RNAseqCounts.txt", "CellLines_ATACseqCounts.txt"),
                        2, 
                        sep="\t", 
                        filtering="stringent")

c_index <- numeric(0)

# For each factorization method
for(i in 1:length(out$factorizations)){
    
    # Get factorization result
    factors <- out$factorizations[[i]][[1]]

    # Delete NAs
    factors <- factors[!is.na(factors[,1]) & !is.na(factors[,2]), ]
    sample_annot <- sample_annot[!is.na(sample_annot[,1]) & !is.na(sample_annot[,2]), ]

    # Data to be plotted
    df <- data.frame(x =  factors[,1], y = factors[,2], color_by = sample_annot[,2])
    # Plot results
    p <- ggplot(df, aes_string(x = "x", y = "y")) + 
       geom_point(aes_string(color = "color_by"), size=dot_size, alpha=dot_alpha) + 
       xlab(xlabel) + ylab(ylabel) +
       # scale_shape_manual(values=c(19,1,2:18)[seq_along(unique(shape_by))]) +
       theme(plot.margin = margin(20, 20, 10, 10), 
             axis.text = element_text(size = rel(1), color = "black"), 
             axis.title = element_text(size = 16), 
             axis.title.y = element_text(size = rel(1.1), margin = margin(0, 10, 0, 0)), 
             axis.title.x = element_text(size = rel(1.1), margin = margin(10, 0, 0, 0)), 
             axis.line = element_line(color = "black", size = 0.5), 
             axis.ticks = element_line(color = "black", size = 0.5),
             panel.border = element_blank(), 
             panel.grid.major = element_blank(),
             panel.grid.minor = element_blank(), 
             panel.background = element_blank(),
             legend.key = element_rect(fill = "white"),
             legend.text = element_text(size = 16),
             legend.title = element_text(size =16)
       )
    p + scale_color_manual(values=c("#0072B2", "#D55E00", "#CC79A7"))
    # Export plot as JPEG image
    ggsave(paste0(results_folder, "plot_",out$method[i],".jpg"))

    # Encode cell type annotations by numeric codes
    ann <- factor(sample_annot[,2], levels=c("HCT", "Hela", "K562"))
    ann <- as.integer(ann)
    # Compare factors and annotations
    c_index <- c(c_index, intCriteria(factors, as.integer(ann), crit=c("C_index"))$c_index)

}

# Build output table
report_cindex <- data.frame(method=out$method, cindex=c_index)

# Export results as one tab-separated table
write.table(report_cindex, file = paste0(results_folder, "singlecell_cindex.txt"), 
            sep="\t", col.names=FALSE, row.names=FALSE, quote=FALSE)


Loading required package: MASS

Loading required package: NMF

Loading required package: pkgmaker

Loading required package: registry

Loading required package: rngtools

Loading required package: cluster

NMF - BioConductor layer [OK] | Shared memory capabilities [NO: bigmemory] | Cores 27/28

  To enable shared memory capabilities, try: install.extras('
NMF
')

Loading required package: mclust

Package 'mclust' version 5.4.6
Type 'citation("mclust")' for citing this R package in publications.

Loading required package: InterSIM

Loading required package: tools

Loading required package: ade4


Attaching package: ‘ade4’


The following object is masked from ‘package:BiocGenerics’:

    score



Attaching package: ‘GPArotation’


The following object is masked from ‘package:NMF’:

    entropy



Attaching package: ‘MOFAtools’


The following objects are masked from ‘package:NMF’:

    featureNames, featureNames<-, predict, sampleNames, sampleNames<-


The following objects are masked f

[1] "No output file provided, using a temporary file..."
Generating warm start... 
K=3:123


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

